# BERT tutorial using Hugging Face
## 教學目標
利用 Hugging Face 套件快速使用 BERT 模型來進行下游任務訓練
- 單一句型分類任務 (single-sentence text classification)

## 適用對象
已經有基本的機器學習知識，且擁有 Python、`numpy`、`pandas`、`scikit-learn` 以及 `PyTorch` 基礎的學生。

若沒有先學過 Python，請參考 [python-入門語法](./python-入門語法.ipynb) 教學。

若沒有先學過 `pandas`，請參考 [pandas-基本功能](./pandas-基本功能.ipynb) 教學。

若沒有先學過 `numpy`，請參考 [numpy-基本功能](./numpy-基本功能.ipynb) 教學。

若沒有先學過 `scikit-learn`，請參考 [scikit-learn-基本功能](./scikit-learn-基本功能.ipynb) 教學。

若沒有先學過  `PyTorch` ，請參考 [PyTorch-基本功能](./PyTorch-基本功能.ipynb) 教學。

若沒有先學過如何使用 `PyTorch` 建立自然語言處理序列模型，請參考 [NN-中文文本分類](./NN-中文文本分類.ipynb) 教學。

## BERT 簡易介紹
### Word embeddings 的問題
![Imgur](https://i.imgur.com/h6U5k41.png)
- 每個單詞的意思在不同的場合下應該有不同的意義表達
- 我們可以利用 RNN 作為語言模型，透過語言模型的輸入與輸出的處理來產生能夠理解上下文語意的 contextual embeddings
    - Language model: 語言模型，藉由估計(或最佳化)一整個序列的生成機率來輸出字詞的模型
        - 可以參考 [language model 的詳細教學](https://youtu.be/LheoxKjeop8?t=50)
- 藉由此種做法，我們可以將單詞語意的 word embeddings 轉換為具有上下文語意的 contextual embeddings

## 所以什麼是 BERT?
- 請參考理論層面的詳細教學 ([影片連結](https://www.youtube.com/watch?v=gh0hewYkjgo))
- 想進行 PyTorch 的 BERT 實作來獲得深入理解可以參考 ([網誌連結](https://leemeng.tw/attack_on_bert_transfer_learning_in_nlp.html))
- 也可以參考 Jay Alammar 的 The Illustrated BERT ([網誌連結](https://jalammar.github.io/illustrated-bert/))
- 也可以參考原始論文 ([論文連結](https://www.aclweb.org/anthology/N19-1423/))

### BERT 的 Pre-training 和 Fine-tuning 與先前方法比較
![Imgur](https://i.imgur.com/qfLhUaG.png)
- Pre-training 已經是 NLP 領域中不可或缺的方法
- 像 BERT 這類基於 Transformers 的[模型非常多](http://speech.ee.ntu.edu.tw/~tlkagk/courses/DLHLP20/BERT%20train%20(v8).pdf)，可以前往 [Hugging Face models](https://huggingface.co/models) 一覽究竟

## Hugging Face 介紹
- 🤗 Hugging Face 是專門提供自然語言處理領域的函式庫
- 其函式庫支援 PyTorch 和 TensorFlow
- 🤗 Hugging Face 的主要套件為:
    1. Transformers ([連結](https://huggingface.co/transformers/index.html))
    - 提供了現今最強大的自然語言處理模型，使用上非常彈性且方便
    2. Tokenizers ([連結](https://huggingface.co/docs/tokenizers/python/latest/))
    - 讓你可以快速做好 BERT 系列模型 tokenization
    3. Datasets ([連結](https://huggingface.co/docs/datasets/))
    - 提供多種自然語言處理任務的資料集

In [ ]:
# 若沒有安裝 transformers 和 datasets 套件，請取消以下註解並執行
# !pip install transformers
# !pip install datasets

In [ ]:
# 1. 確認所需套件的版本

import torch
print("PyTorch 的版本為: {}".format(torch.__version__))

import transformers
print("Hugging Face Transformers 的版本為: {}".format(transformers.__version__))

import datasets
print("Hugging Face Datasets 的版本為: {}".format(datasets.__version__))

In [ ]:
# 2. 載入其他所需套件

import os
import json
import numpy as np
from pathlib import Path # (Python3.4+)

# 單一句型分類任務 (single-sentence text classification)
## 準備資料集 (需先下載)
我們使用 IMDb reviews 資料集作為範例

In [ ]:
# 若沒有安裝 wget 套件，請取消以下註解並執行
# !pip install wget

In [ ]:
# 3. 下載 IMDb 資料集

import wget
url = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
filename = wget.download(url, out='./')

In [ ]:
# 若沒有安裝 tarfile 套件，請取消以下註解並執行
# !pip install tarfile

In [ ]:
# 4. 解壓縮 IMDb 資料集

import tarfile

# 指定檔案位置，並解壓縮 .gz 結尾的壓縮檔
tar = tarfile.open('aclImdb_v1.tar.gz', 'r:gz')
tar.extractall()

## 接下來我們要進行資料前處理
但首先要觀察解壓縮後的資料夾結構:
```
aclImdb---
        |--train
        |    |--neg
        |    |--pos
        |    |--...
        |--test
        |    |--neg
        |    |--pos
        |    |--...
        |--imdb.vocab
        |--imdbEr.text
        |--README
```
其中 train 和 test 資料夾中分別又有 neg 和 pos 兩種資料夾

我們要針對這兩個目標資料夾進行處理

In [ ]:
# 5. 前處理 IMDb 資料 (定義 function)
def read_imdb_split(split_dir):
    """針對 IMDb 資料集進行讀檔及正負向歸類
    Args:
        - split_dir: IMDb 資料集的資料夾路徑
    Return:
        - texts: 資料集的語句部分
        - labels: 資料集的標籤部分
    """
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        # 利用iterdir() 來列出資料夾底下的所有檔案，此功能等同於 os.path.listdir()
        # 使用 glob 的語法分取得副檔名為 .txt 的檔案
        for text_file in (split_dir/label_dir).glob("*.txt"):
            # read_text() 是 Pathlib的好用功能
            tmp_text = text_file.read_text()
            # 將讀到的文字 append 到我們事先定義的 list 中
            texts.append(tmp_text)
            # 將資料夾標籤作為 label，並 append 到我們事先定義的 list 中
            labels.append(0 if label_dir == "neg" else 1)
    
    return texts, labels

In [ ]:
# 6. 前處理 IMDb 資料 (執行)

train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')

### 切分訓練資料，來分出 validation set

In [ ]:
# 7. 使用 train_test_split 來切出 validation set

from sklearn.model_selection import train_test_split

# 設立隨機種子來控制隨機過程
random_seed = 42

# 設定要分出多少比例的 validation data
valid_ratio = 0.2

# 使用 train_test_split 來切分資料
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, 
    train_labels,
    test_size=valid_ratio, 
    random_state=random_seed
)

## 輸入 BERT 的前處理
![Imgur](https://i.imgur.com/3C7xDlf.png)
(圖片來源: BERT [原始論文](https://www.aclweb.org/anthology/N19-1423/))

### Tokenization
- 斷字的部份以 DistilBERT (Sanh et al., 2019) 的 tokenizer 為例
- Hugging Face 的 tokenizer 可以直接幫你自動將資料轉換成 BERT 的輸入型式 (也就是加入[CLS]和[SEP] tokens)

## Hugging Face AutoTokenizer
- 使用 AutoTokenizer 搭配 Hugging Face models 的名稱可以直接呼叫使用
- 舉例:
    - transformers.AutoTokenizer.from_pretrained('distilbert-base-uncased')
    - 等同於 transformers.DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
- [點這裡來查看Hugging Face models 的名稱](https://huggingface.co/transformers/pretrained_models.html)

In [ ]:
# 8. 載入 tokenizer

# 在 Hugging Face 套件中可使用 .from_pretrained() 的方法來導入預訓練模型
tokenizer = transformers.AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
# 9. 分別將3種資料 (train/valid/test) 做 tokenization
# truncation 代表依照 max_length 進行序列長度的裁切
# max_length 可以在 tokenizer 的 parameters 中進行設定
# 如果沒有指定 max_length，則依照所使用的模型的序列最大長度
# padding 為 True 表示會將序列長度補齊至該 batch 的最大長度 (欲知詳情請查看 source code)

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
# 10. 查看 max_length

tokenizer.model_max_length

In [ ]:
# 11. 查看 [CLS] token 和 [SEP] token 在字典中的 ID

print("The ID of [CLS] token is {}.".format(tokenizer.vocab["[CLS]"]))
print("The ID of [SEP] token is {}.".format(tokenizer.vocab["[SEP]"]))

### 檢查 tokenization 後的結果
- 使用 Hugging Face tokenizer 進行 tokenization 後的結果是一個 dict
- 這個 dict 的 keys 包含 'input_ids' 和 'attention_mask'
- input_ids: 原本句子中的每個字詞被斷詞後轉換成字典的 ID
    - 注意!! tokenizer 小小的動作已經幫你完成了斷詞和 word to ID 的轉換
- attention_mask: tokenization 後句子中包含文字的部分為 1，padding 的部分為 0
    - 可以想像成模型需要把注意力放在有文字的位置

In [ ]:
# 12. 檢查 tokenization 後的結果

print(val_encodings.keys())
print(val_encodings.input_ids[0])
print(val_encodings.attention_mask[0])

In [ ]:
# 13. 透過 PyTorch Dataset 來建立能夠進行方便資料存取的格式

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        # Dataset class 的 parameters 放入我們 tokenization 後的資料以及資料的標籤
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # 請注意 tokenization 後的資料是一個 dict
        # 在此步驟將資料以及標籤都轉換為 PyTorch 的 tensors
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])

        return item

    def __len__(self):
        # 回傳資料集的總數
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

### 除了自己處理資料，你還可以使用 Hugging Face Datasets
- Hugging Face Datasets 已經幫你收錄了自然語言處理領域常見的資料集
- 直接呼叫 Datasets 並搭配下面幾個 cells 的語法，可省下不少時間
- 但前提是你要進行的任務資料集有被收錄在 Hugging Face Datasets

In [ ]:
# 14. 查看 Hugging Face Datasets 的資訊

datasets_list = datasets.list_datasets()

print("現在 Hugging Face Datasets 有 {} 個資料集可以使用".format(len(datasets_list)))
print("===============================================")
# print("所有的資料集如下: ")
# print(', '.join(dataset for dataset in datasets_list))

In [ ]:
# 15. 從 Hugging Face Datasets 載入資料並做資料切分

# 載入 IMDb 的訓練資料集
train = datasets.load_dataset("imdb", split="train")

# 設立隨機種子來控制隨機過程
random_seed = 42
# 從 IMDb 的訓練資料集中切分出驗證資料集
splits = train.train_test_split(
    test_size=0.2,
    seed=random_seed
)
train, valid = splits['train'], splits['test']

# 載入 IMDb 的測試資料集
test = datasets.load_dataset("imdb", split="test")

In [ ]:
print(len(train))
print(len(valid))
print(len(test))

In [ ]:
# 16. 將 Hugging Face Datasets 轉為 PyTorch Dataset 的封裝

def to_torch_data(hug_dataset):
    """將 Hugging Face Datasets 轉為 PyTorch Dataset
    Args:
        - hug_dataset: 從 Datasets 載入的資料集
    Return:
        - dataset: 已轉為 PyTorch Dataset 的資料集
    """
    dataset = hug_dataset.map(
        lambda batch: tokenizer(
            batch["text"],
            truncation=True,
            padding=True
        ),
        batched=True
    )
    dataset.set_format(
        type='torch',
        columns=[
            'input_ids',
            'attention_mask',
            'label'
        ]
    )
    return dataset

train_dataset = to_torch_data(train)
val_dataset = to_torch_data(valid)
test_dataset = to_torch_data(test)

## 進行模型的訓練
### 使用 Hugging Face Trainer ([Documentation](https://huggingface.co/transformers/main_classes/trainer.html))
- Trainer 是 Hugging Face 中高度封裝的套件之一，負責模型訓練時期的"流程"
- 過去我們自行寫訓練流程的程式碼可以交給 Trainer
- Trainer 需要搭配使用 [TrainingArguments](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments)
    - TrainingArguments 是 Trainer 所需要的引數

In [ ]:
# print(train_dataset[0])

In [ ]:
# 17. 建立自定的評估的指標 (定義 function)
# 將作為 transformers.Trainer 的 parameters 之一

# Scikit-learn 的 precision_recall_fscore_support 套件可以一次計算 F1 score, precision, 和 recall
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
# 18. 訓練模型

# 設定 TrainingArguments
training_args = transformers.TrainingArguments(
    output_dir='./results',          # 輸出的資料夾
    num_train_epochs=3,              # 總共訓練的 epoch 數目
    per_device_train_batch_size=16,  # 訓練模型時每個裝置的 batch size
    per_device_eval_batch_size=64,   # 驗證模型時每個裝置的 batch size
    warmup_steps=500,                # learning rate scheduler 的參數
    weight_decay=0.01,               # 最佳化演算法 (optimizer) 中的權重衰退率
    logging_dir='./logs',            # 存放 log 的資料夾
    logging_steps=10,
    seed=random_seed
)

# 利用 AutoModel 呼叫模型
model = transformers.AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = transformers.Trainer(
    model=model,                         # 🤗 的模型
    args=training_args,                  # Trainer 所需要的引數
    train_dataset=train_dataset,         # 訓練集 (注意是 PyTorch Dataset)
    eval_dataset=val_dataset,            # 驗證集 (注意是 PyTorch Dataset)，可使 Trainer 在進行訓練時也進行驗證
    compute_metrics=compute_metrics      # 自定的評估的指標
)

# 指定使用 1 個 GPU 進行訓練
trainer.args._n_gpu=1

# 開始進行模型訓練
trainer.train()

In [ ]:
# 19. 測試模型

trainer.predict(test_dataset)